<a href="https://colab.research.google.com/github/Harold-Rodriguez/Visualisation/blob/master/Project_ECG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn import ensemble
from pandas import read_csv
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from pandas import read_csv
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

In [ ]:
from siml.sk_utils import *
from siml.signal_analysis_utils import *

import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict, Counter

In [ ]:
with h5py.File("ecg_tracings.hdf5", "r") as f:
    x = np.array(f['tracings'])

y=pd.read_csv("gold_standard.csv")

In [ ]:
x_flatten=x.reshape(827,12*4096)
X_Train, X_Test, Y_Train, Y_Test = train_test_split(x_flatten, y["1dAVb"], test_size=0.20, random_state=1) 

In [ ]:
N = 49152
f_s = 200
t_n = 2.56
T = t_n / N
#amplitudes = X_Train
#x_value = np.linspace(0,t_n,N)
#frequencies = [17, 15.5, 14, 12.5, 11, 9.5, 8, 6.5, 5, 3, 1.5, 1]
#y_values = [amplitudes[ii]*np.sin(2*np.pi*frequencies[ii]*x_value) for ii in range(0,len(amplitudes))]
#composite_y_value = np.sum(y_values, axis=0)

In [ ]:
def get_fft_values(y_values, T, N, f_s):
    f_values = np.linspace(0.0, 1.0/(2.0*T), N//2)
    fft_values_ = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values_[0:N//2])
    return f_values, fft_values

def get_psd_values(y_values, T, N, f_s):
    f_values, psd_values = welch(y_values, fs=f_s)
    return f_values, psd_values

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[len(result)//2:]
 
def get_autocorr_values(y_values, T, N, f_s):
    autocorr_values = autocorr(y_values)
    x_values = np.array([T * jj for jj in range(0, N)])
    return x_values, autocorr_values

In [ ]:
def get_first_n_peaks(x,y,no_peaks=5):
    x_, y_ = list(x), list(y)
    if len(x_) >= no_peaks:
        return x_[:no_peaks], y_[:no_peaks]
    else:
        missing_no_peaks = no_peaks-len(x_)
        return x_ + [0]*missing_no_peaks, y_ + [0]*missing_no_peaks
    
def get_features(x_values, y_values, mph):
    indices_peaks = detect_peaks(y_values, mph=mph)
    peaks_x, peaks_y = get_first_n_peaks(x_values[indices_peaks], y_values[indices_peaks])
    return peaks_x + peaks_y

def extract_features_labels(dataset, labels, T, N, f_s, denominator):
    percentile = 5
    list_of_features = []
    list_of_labels = []
    for signal_no in range(0, len(dataset)):
        features = []
        list_of_labels.append(labels[signal_no])
        for signal_comp in range(0,dataset.shape[0]):
            signal = dataset[signal_no,:]
            
            signal_min = np.nanpercentile(signal, percentile)
            signal_max = np.nanpercentile(signal, 100-percentile)
            ijk = (100 - 2*percentile)/10
            mph = signal_min + (signal_max - signal_min)/denominator
            
            features += get_features(*get_psd_values(signal, T, N, f_s), mph)
            features += get_features(*get_fft_values(signal, T, N, f_s), mph)
            features += get_features(*get_autocorr_values(signal, T, N, f_s), mph)
        list_of_features.append(features)
    return np.array(list_of_features), np.array(list_of_labels)

In [ ]:
denominator = 10
X_train, Y_train = extract_features_labels(X_Train, Y_Train, T, N, f_s, denominator)
X_test, Y_test = extract_features_labels(X_Test, Y_Test, T, N, f_s, denominator)

KeyError: 3